In [20]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score

In [21]:
ytrue = pd.read_csv('./data/res_baseline.csv', sep=",", index_col=0)

In [22]:
ytruec = ytrue['classification']
ytruer = ytrue['regression']

In [23]:
data = pd.read_csv('./data/train_series.csv', sep=",", index_col=0)
data['Index + 1'][data['Index + 1'] >= 0] = 1
data['Index + 1'][data['Index + 1'] < 0] = 0

C:\Users\lucas\AppData\Local\Temp\ipykernel_8380\604869430.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Index + 1'][data['Index + 1'] >= 0] = 1
C:\Users\lucas\AppData\Local\Temp\ipykernel_8380\604869430.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Index + 1'][data['Index + 1'] < 0] = 0


In [24]:
data

,Index Name,Index - 9,Index - 8,Index - 7,Index - 6,Index - 5,Index - 4,Index - 3,Index - 2,Index - 1,Index - 0,index ecb,index fed,Index + 1
0,V2X Index,0.001045,0.005841,0.003832,-0.027519,-0.103565,-0.045086,-0.011265,0.005164,0.054050,0.015779,"1126,1471",96,1.0
1,MOVE Index,-0.021497,0.007891,-0.013175,-0.008436,0.000000,0.026303,0.000556,0.001455,0.007422,0.000000,975,568,1.0
2,SPX Index,-0.001872,-0.008154,0.023588,0.004086,0.003493,0.003300,0.000885,-0.011304,0.005040,0.000156,"78,806","328,695",1.0
3,SPX Index,0.004980,-0.000864,0.001677,0.000000,0.006030,-0.001083,0.000419,0.001492,0.001018,-0.002582,"560,1405",118,1.0
4,EURUSD Curncy,0.000360,-0.001893,0.005579,-0.003056,-0.001171,-0.001623,-0.002350,-0.006444,-0.000729,-0.000365,404,261,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8941,SPX Index,-0.002045,-0.003642,0.008096,0.004949,-0.001264,0.000150,0.002482,0.000000,-0.000786,-0.002723,294,195,0.0
8942,SX5E Index,-0.027361,0.094386,0.002350,-0.005076,0.022555,0.000963,-0.060170,0.028323,0.006516,-0.000338,277,"199,372",0.0
8943,SX5E Index,-0.008359,0.021658,-0.015986,-0.023528,0.020072,0.017822,0.005648,0.017045,0.008557,-0.003072,"78,806","99,625",1.0
8944,SRVIX Index,0.011945,0.002567,0.000000,0.005115,0.000364,0.003999,0.012978,0.012458,-0.006032,0.005088,"1660,20",79,0.0


In [25]:
X = data.drop(columns=['Index + 1', 'index ecb', 'index fed'])
y = data['Index + 1']
y.value_counts()


1.0    4722
0.0    4224
Name: Index + 1, dtype: int64

In [26]:
X_train = X.iloc[0:5000]
X_test = X.iloc[5000:X.shape[0]]
y_train = y.iloc[0:5000]
y_test = y.iloc[5000:X.shape[0]]

In [27]:
ytruer = ytruec.iloc[5000: X.shape[0]]
ytruer

5000    1
5001    1
5002    0
5003    1
5004    1
       ..
8941    0
8942    1
8943    1
8944    1
8945    0
Name: classification, Length: 3946, dtype: int64

In [28]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Index Name  5000 non-null   object 
 1   Index - 9   5000 non-null   float64
 2   Index - 8   5000 non-null   float64
 3   Index - 7   5000 non-null   float64
 4   Index - 6   5000 non-null   float64
 5   Index - 5   5000 non-null   float64
 6   Index - 4   5000 non-null   float64
 7   Index - 3   5000 non-null   float64
 8   Index - 2   5000 non-null   float64
 9   Index - 1   5000 non-null   float64
 10  Index - 0   5000 non-null   float64
dtypes: float64(10), object(1)
memory usage: 468.8+ KB


In [29]:
cat = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    cat_features=['Index Name'],
    loss_function='Logloss',
    logging_level='Verbose',
    allow_writing_files=False
)
cat.fit(
    X=X_train,
    y=y_train,
    eval_set=(X_test,y_test),
    verbose = 10
)

0:	learn: 0.6891289	test: 0.6903531	best: 0.6903531 (0)	total: 7.12ms	remaining: 705ms
10:	learn: 0.6673433	test: 0.6755285	best: 0.6755285 (10)	total: 65.9ms	remaining: 533ms
20:	learn: 0.6568319	test: 0.6722385	best: 0.6722385 (20)	total: 127ms	remaining: 476ms
30:	learn: 0.6491438	test: 0.6709239	best: 0.6709239 (30)	total: 185ms	remaining: 412ms
40:	learn: 0.6445570	test: 0.6707942	best: 0.6705729 (35)	total: 252ms	remaining: 363ms
50:	learn: 0.6394253	test: 0.6704481	best: 0.6704083 (48)	total: 320ms	remaining: 307ms
60:	learn: 0.6346275	test: 0.6706601	best: 0.6702575 (56)	total: 384ms	remaining: 246ms
70:	learn: 0.6276653	test: 0.6703310	best: 0.6701832 (69)	total: 459ms	remaining: 187ms
80:	learn: 0.6237361	test: 0.6700700	best: 0.6698843 (76)	total: 528ms	remaining: 124ms
90:	learn: 0.6194039	test: 0.6701243	best: 0.6698843 (76)	total: 595ms	remaining: 58.8ms
99:	learn: 0.6151795	test: 0.6705688	best: 0.6698843 (76)	total: 648ms	remaining: 0us

bestTest = 0.6698842905
bestIter

In [30]:
yp = cat.predict(X_test)

In [31]:
yp = pd.Series(yp)
yp

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
3941    1.0
3942    1.0
3943    1.0
3944    1.0
3945    1.0
Length: 3946, dtype: float64

In [32]:
ytruer

5000    1
5001    1
5002    0
5003    1
5004    1
       ..
8941    0
8942    1
8943    1
8944    1
8945    0
Name: classification, Length: 3946, dtype: int64

In [33]:
a = accuracy_score(yp, y_test)
b = accuracy_score(ytruer, y_test)
c = f1_score(yp, y_test)
d = f1_score(ytruer, y_test)
e = recall_score(yp, y_test)
f = recall_score(ytruer, y_test)
g = roc_auc_score(yp, y_test)
h = roc_auc_score(ytruer, y_test)

In [34]:
res = np.array([[a,b],[c,d],[e,f],[g,h]])
resd = pd.DataFrame(res)
resd

,0,1
0,0.534719,0.467816
1,0.618612,0.471033
2,0.534458,0.481462
3,0.534901,0.468027
